# Dataset generation

## 1. Import

In [3]:
import numpy as np
import networkx as nx

## 2. Generating

In [4]:
#N = 1000    # number of samples
#M = 20      # Number of nodes

#X = np.random.uniform(0,1, size=(N, M, M))

#X = np.triu(X)
#for i in range(X.shape[0]):
#    X[i,:] = X[i,:] + X[i,:].T
#    X[np.concatenate(i,np.diag_indices(M))] = .0
#print(X[0])

In [8]:
nx.generators.empty_graphs(10)

AttributeError: module 'networkx.generators' has no attribute 'empty_graphs'